CLEMnet: Testing the model
====================

This notebook is used to test CLEMnet.

It is divided into 5 parts:
- Initialising (importing packages, setting up GPU environment, etc.)
- Model (import model and weights, set up metrics, etc.)
- DataGenerators and evaluation (set up data generators and use them to evaluate model with and without edge-pixel correction)
- Prediction (create one large prediction of entire dataset)
- Catmaid (use catmaid to overlay and compare EM image with prediction and true FM image)

________________
Initialising
---------

- Importing all desired packages. 
- Setting up GPU and CPU environment. 
- Initialising general variables. 

In [6]:
##### Libary #####
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.keras import backend as K

import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import datetime

import clem_net as clem
import catmaid_v10_v2 as catmaid

%matplotlib inline

In [7]:
##### GPU setup #####
os.environ["CUDA_VISIBLE_DEVICES"]="1"    #chose which GPU to use (0-3)

##### CPU setup #####
os.environ['MKL_NUM_THREADS'] = '20'     #chose the amount of CPUs to use, idk if this is working right now?
os.environ['GOTO_NUM_THREADS'] = '20'    #however, maybe works if GPU is not present/working
os.environ['OMP_NUM_THREADS'] = '20'
os.environ['openmp'] = 'True'

In [8]:
##### General variables #####
training_dir = '/home/rlane/FMML_DATA/20200618_RL012/'     #directory to find trainable tiles
EM_tile_size = 1024     #size in pixels of 1 EM tile
FM_tile_size = 1024     #size in pixels of 1 FM tile
EMFM_lvl_d = 5          #difference in scale between EM and FM
augmen = True           #augmentation on (True) or off (False)
edge_pixels = 4         #amount of pixels affected by convolutional edge effect

clem.variables(training_dir, EM_tile_size, FM_tile_size, EMFM_lvl_d, augmen)

##### Name of weights/model to test on #####
weight_name = '134910_hoechst_2us'

General variables loaded


_______________
Initialize and load model
------------



In [9]:
##### set up metrics #####
def acc_pred(y_true, y_pred):
    'accuracy metric as described in progress report'
    a = (1-abs(y_true-y_pred))
    b = keras.backend.sum(a)/(32*32)/batch_size
    return b

def pearson(y_true, y_pred):
    'pearson correlation coefficient'
    x0 = y_true-K.mean(y_true)
    y0 = y_pred-K.mean(y_pred) 
    return K.sum(x0*y0) / (K.sqrt(K.sum((x0)**2)*K.sum((y0)**2))+1e-10)

batch_size = 1

In [15]:
##### Load model for evaluation ####
## Load model architecture
model = clem.getModel()
model.compile(optimizer=tf.keras.optimizers.Adam(lr=1e-5), loss='binary_crossentropy', metrics=[acc_pred, pearson])
#model.summary()

## load model weights ###
name_weight = "weights/" + weight_name + ".h5"
model.load_weights(name_weight)

##### Load model for prediction ####
## Load model architecture
model_pred = clem.getModel_pred()
model_pred.compile(optimizer=tf.keras.optimizers.Adam(lr=1e-5), loss='binary_crossentropy', metrics=[acc_pred, pearson])
#model.summary()

## load model weights
name_weight = "weights/" + weight_name + ".h5"
model_pred.load_weights(name_weight)

___________
Set up DataGenerators and evaluate model
---------
- The dataset which will be tested is selected
- Two DataGenerators are built: one with and one without edge-pixel correction
- Model is evaluate on test dataset

In [ ]:
##### Locate all datasets #####
#DNA/RNA (hoechst)
section_subfolders = [{"EM": "2us/lil_EM_2us_montaged/1/", "FM": "2us/hoechst_correlated/1/"}, 
                   {"EM": "2us/lil_EM_2us_montaged/3/", "FM": "2us/hoechst_correlated/3/"},
                   {"EM": "2us/lil_EM_2us_montaged/4/", "FM": "2us/hoechst_correlated/4/"},
                   {"EM": "2us/lil_EM_2us_montaged/7/", "FM": "2us/hoechst_correlated/7/"},
                   {"EM": "2us/lil_EM_2us_montaged/9/", "FM": "2us/hoechst_correlated/9/"},
                   {"EM": "2us/lil_EM_2us_montaged/10/", "FM": "2us/hoechst_correlated/10/"}]

##### Select test dataset(s) #####
folder_selec = i
testsection_subfolders = [section_subfolders[folder_selec]]
print('Testing will be performed on dataset:')
for x in testsection_subfolders:
    print(x)

##### Combining all functions concerning patches into one class #####
test_sections = clem.getSectionList(testsection_subfolders, clem.training_dir)

In [ ]:
##### Set up Generators #####
evaluation_generator = clem.TestGenerator(test_sections, batch_size, EM_coords = (0,0), edge_pixels=4, evaluation=True)
prediction_generator = clem.TestGenerator(test_sections, batch_size, EM_coords = (0,0), edge_pixels=4, evaluation=False)

In [ ]:
##### Evaluate model (metrics)#####
results_eval = model.evaluate_generator(evaluation_generator, verbose=1)
results_pred = model_pred.evaluate_generator(prediction_generator, verbose=1)

___________
Create prediction
--------
- Create prediction of all available EM patches 
- However, make ever EM patch edge_pixels larger to account for convolutional edge effect
- Stitch all predictions together, but delete edge_pixels amount of edge pixels on every side of every prediction first.
- Save prediction as numpy array to maintain resolution

In [ ]:
##### Create predictions #####
predictions = model_pred.predict_generator(prediction_generator, verbose=1)

In [ ]:
##### Glue predictions #####

# Initialization
cut_off = np.array(test_sections[0].max_tiles)*32+64    #finding length and width of total area
fm_pred = np.zeros(((cut_off[0]), (cut_off[1])))
em_set = test_sections[0].sectionXYList

# Predict FM
for k, im_FM_patch in enumerate(predictions):
    tile_xy = em_set[k]
    i = int(tile_xy[0])*32
    j = int(tile_xy[1])*32
    fm_pred[i:i+32, 
            j:j+32] = im_FM_patch[:,:,0]

In [ ]:
#### Show and save prediction #####

# Figure
fig_size = 8
fig = plt.figure(figsize=(fig_size,fig_size))
plt.imshow(fm_pred)
plt.axis('off')

# Save both as figure and (more important) as .npy file
# fig.savefig('figures/' + weight_name + '_hoechst_' + str(folder_selec) + '_2us.png')
file_name = testsection_subfolders[0]["EM"].split('/')
prediction_name = 'predictions/' + weight_name + '/' + file_name[1] + '_' + file_name[2]
np.save(prediction_name, fm_pred)

In [96]:
##### load predictions #####
#In case of already existing prediction, call that prediction in the next line
#prediction_name = 'predictions/' + '00_FINAL_13479_hoechst_data_aug_2_hoechst_5_2us'
predictions = np.load(prediction_name + '.npy')

___________
Catmaid, show predictions
----------
- Use catmaid to compare predictions with EM and true FM images.
- Here, a simplified version of catmaid is used, for more possibilities, use `image_viewer`

In [13]:
catmaid.variables(training_dir, EM_tile_size, FM_tile_size, EMFM_lvl_d)

General variables loaded


In [14]:
fig_size = 9
fig_name = weight_name + '_hoechst_' + str(folder_selec)
catmaid.catmaid(predictions, testsection_subfolders, fig_size, fig_name)

Button(description='Save', style=ButtonStyle())

Output()

Output()